In [1]:
import json
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
import inflect
import time
import gensim
engine = inflect.engine()

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
recipes = json.load(open('../generated/all_recipes_repr.json'))

In [3]:
len(recipes)

963396

In [4]:
def singularize(word):
    ingr = engine.singular_noun(word)
    return word if (not ingr) else ingr

def clean_ing_word(word) : return singularize(word).lower()

def clean_whole_ing(ing) : return " ".join([clean_ing_word(word) for word in ing.split(" ")])

Word2Vec training

In [5]:
start_time = time.time()
model = gensim.models.Word2Vec(recipes, min_count=2)

time_after_creation = time.time()
print("creation of model :", time_after_creation - start_time, "sec.")

model.train(recipes, total_examples=len(recipes), epochs=10)

time_after_training = time.time()
print("model training :", time_after_training - time_after_creation, "sec.")

model.save('../generated/food_embeddings.bin')

time_after_saving = time.time()
print("model saving :", time_after_saving - time_after_training, "sec.")

creation of model : 11.180386304855347 sec.
model training : 20.061383962631226 sec.
model saving : 0.017952680587768555 sec.


Results investigation

In [6]:
model.wv.most_similar("lamb")

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lean lamb', 0.8576267957687378),
 ('lamb shoulder', 0.7550902366638184),
 ('lamb chop', 0.6110999584197998),
 ('shoulder lamb chop', 0.5982378721237183),
 ('lamb shank', 0.5843186974525452),
 ('lamb rib chop', 0.5696008205413818),
 ('eggplant', 0.5672318935394287),
 ('mutton', 0.5672123432159424),
 ('loin lamb', 0.5623551607131958),
 ('goat meat', 0.5289875864982605)]

We see that the adjectives are creating new ingredients. Given we are only interested in flavours associations, we should ignore them. Indeed, shredded mushrooms and dried mushrooms should have the same flavour. 

In [7]:
model.wv.similarity(w1="salt", w2="pepper")

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.4761162

In [8]:
 model.most_similar(positive=['rice', 'tomato'], negative=['pasta'], topn=1)

C:\Users\Martin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('beef tongue', 0.4903402328491211)]